<a href="https://colab.research.google.com/github/ryyutku/DSGP/blob/anuk/Modelling/Demand_forecast_model_6_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Training model withoutdifferencing or any other augmentation and transformations**

In [75]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [76]:
df = pd.read_csv('CIEC.csv')

In [77]:
df.columns

Index(['date', 'fuel_consumption', 'petroleum_imports_crudeOil',
       'Taxes_on_Customs_and_Other_Import Duties ',
       'Foreign Direct Investments', 'GDP Goods and Services',
       'New Vehicle Registrations', 'Vehicle Sales', 'Vehicle Sales Asia',
       'No.of Vessels Colombo', 'Imports of Refined Products',
       'Tax income profits_gains', 'Tax Goods & Services',
       'Tax Road Transport', 'GDP FCE Households', 'Diesel User Price',
       'Petrol User Price', 'Consumption_Oil', 'Sales 90 Octane',
       'Sales 95 Octane', 'Sales Auto Diesel'],
      dtype='object')

In [78]:
df['date'] = pd.to_datetime(df['date'])

In [79]:
features = ['date','fuel_consumption', 'petroleum_imports_crudeOil',
       'Taxes_on_Customs_and_Other_Import Duties',
       'Foreign Direct Investments', 'GDP Goods and Services',
       'GDP: Gross National Income', 'Government Debt',
       'New Vehicle Registrations', 'Vehicle Sales', 'Port Stay Duration',
       'Vehicle Sales Asia', 'No.of Vessels Colombo',
       'Imports of Refined Products', 'Colombo port calls',
       'Tax income profits_gains', 'Tax on Export', 'Tax Goods & Services',
       'Tax Road Transport', 'GDP FCE Households', 'Diesel User Price',
       'Petrol User Price', 'Consumption_Oil', 'Sales 90 Octane',
       'Sales 95 Octane', 'Sales Auto Diesel', 'Household_income',
       'Fuel_other_manufacture']

In [80]:
df_original = df.copy()

In [81]:
for feature in df.columns:
  print("fuel consumption and",feature," ",end="")
  print(df['fuel_consumption'].corr(df[feature]))

fuel consumption and date  0.6471771865361186
fuel consumption and fuel_consumption  1.0
fuel consumption and petroleum_imports_crudeOil  0.7749175343518945
fuel consumption and Taxes_on_Customs_and_Other_Import Duties   0.23858656484644236
fuel consumption and Foreign Direct Investments  0.49341996546192624
fuel consumption and GDP Goods and Services  0.7256692082264404
fuel consumption and New Vehicle Registrations  0.4020013199119769
fuel consumption and Vehicle Sales  0.773305958286014
fuel consumption and Vehicle Sales Asia  0.2252780043781615
fuel consumption and No.of Vessels Colombo  0.19077249280425274
fuel consumption and Imports of Refined Products  0.5899575275082887
fuel consumption and Tax income profits_gains  0.5193520756044036
fuel consumption and Tax Goods & Services  -0.4789936968485389
fuel consumption and Tax Road Transport  0.7682688182001607
fuel consumption and GDP FCE Households  0.6213174075360737
fuel consumption and Diesel User Price  0.2489837325220658
fuel

## **Checking the time frame with the most columns available**

In [82]:
def find_best_timeframe(df, time_column):
  df = df.copy()

  df_sorted = df.sort_values(by=time_column).set_index(time_column)

  # non-null values per row
  non_null_counts = df_sorted.notnull().sum(axis=1)

  # Finding the best periods with max non-null values
  best_start, best_end = None, None
  max_columns = 0

  current_start = df_sorted.index[0]
  current_max = non_null_counts.iloc[0]

  for i in range(1, len(non_null_counts)):
    if non_null_counts.iloc[i] >= max_columns:
      max_columns = non_null_counts.iloc[i]
      best_start = current_start
      best_end = df_sorted.index[i]
    else:
      current_start = df_sorted.index[i]

  return best_start, best_end

In [83]:
start,end = find_best_timeframe(df, 'date')
print(start, end)

2009-12-28 00:00:00 2014-12-29 00:00:00


In [84]:
df = df[(df['date']>=start) & (df['date']<=end)]

In [85]:
print(df.isnull().sum())

date                                         0
fuel_consumption                             0
petroleum_imports_crudeOil                   0
Taxes_on_Customs_and_Other_Import Duties     0
Foreign Direct Investments                   0
GDP Goods and Services                       1
New Vehicle Registrations                    0
Vehicle Sales                                0
Vehicle Sales Asia                           0
No.of Vessels Colombo                        0
Imports of Refined Products                  0
Tax income profits_gains                     0
Tax Goods & Services                         0
Tax Road Transport                           0
GDP FCE Households                           1
Diesel User Price                            0
Petrol User Price                            0
Consumption_Oil                              0
Sales 90 Octane                              0
Sales 95 Octane                              0
Sales Auto Diesel                            0
dtype: int64


In [86]:
len(df.columns)

21

In [87]:
from scipy.stats import boxcox
import numpy as np
import pandas as pd

import pandas as pd
import numpy as np
from scipy.stats import boxcox

def rolling_stats_transformation(df, target_col='fuel_consumption', date_col='date'):
    df = df.copy()

    # Ensure 'date' is in datetime format
    df[date_col] = pd.to_datetime(df[date_col])

    # Sort by date to avoid accidental data leakage
    df = df.sort_values(by=date_col)

    # Set 'date' column as index
    df = df.set_index(date_col)

    # 1️ **Rolling Statistics Features (Using Only Past Data)**
    df['rolling_mean_7'] = df[target_col].shift(1).rolling(window=7, min_periods=1).mean()
    df['rolling_std_7'] = df[target_col].shift(1).rolling(window=7, min_periods=1).std()
    df['rolling_median_7'] = df[target_col].shift(1).rolling(window=7, min_periods=1).median()

    # 2️⃣ **Exponential Moving Average (EMA) (No Future Values)**
    df['ema_0.1'] = df[target_col].shift(1).ewm(alpha=0.1, adjust=False).mean()
    df['ema_0.3'] = df[target_col].shift(1).ewm(alpha=0.3, adjust=False).mean()

    # 3️⃣ **Time-Based Features**
    df['dayofweek'] = df.index.dayofweek
    df['month'] = df.index.month
    df['quarter'] = df.index.quarter
    df['is_weekend'] = df['dayofweek'].isin([5, 6]).astype(int)  # 1 if weekend, 0 otherwise

    # 4️⃣ **Target Variable Transformation (Ensure No Leakage)**
    df['log_fuel_consumption'] = np.log1p(df[target_col])  # Log transformation
    df['boxcox_fuel_consumption'], lambda_bc = boxcox(df[target_col] + 1)  # Box-Cox transformation

    # 5️⃣ **Interaction Terms**
    if 'fuel_price' in df.columns:  # Example: Check if 'fuel_price' exists in the dataset
        df['price_per_demand'] = df['fuel_price'] / (df[target_col] + 1)  # Avoid division by zero

    return df




In [88]:
df = rolling_stats_transformation(df)

In [89]:
drop = ['rolling_mean_7','ema_0.3','GDP Goods and Services']

In [90]:
df = df.drop(drop,axis=1)

# Modelling

In [91]:
df.dropna(inplace=True)

In [92]:
# # Extracting date feature
# df['year'] = df['date'].dt.year
# df['month'] = df['date'].dt.month
# df['day'] = df['date'].dt.day
# df['weekday'] = df['date'].dt.weekday
# df['quarter'] = df['date'].dt.quarter

# df.drop('date',axis=1,inplace=True)

## **Scaling**

In [93]:
# Feature scaling
numerical_cols = df.select_dtypes(include=['float64','int64']).columns
numerical_cols = numerical_cols.drop('fuel_consumption')

# scaler = StandardScaler()
# df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# fuel_scaler = StandardScaler()
# df['fuel_consumption'] = fuel_scaler.fit_transform(df[['fuel_consumption']])

# df.head()

In [94]:
# splitting data into feature and target variables
X = df.drop('fuel_consumption',axis=1)
y = df['fuel_consumption']

In [95]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state= 42)
print("Training shape",X_train.shape)
print("Test df shape",X_test.shape)

Training shape (208, 27)
Test df shape (52, 27)


In [96]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
import plotly.express as px

In [97]:
# Initialize the model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_rf = rf_model.predict(X_test)


In [98]:
# Using Random Forest (replace with y_pred_lr for Linear Regression)
predictions = y_pred_rf  # Replace with y_pred_lr if you're using Linear Regression

In [99]:
from sklearn.model_selection import cross_val_score
# Perform 5-Fold Cross-Validation
scores = cross_val_score(rf_model, X, y, cv=5, scoring='r2')  # Using R² as the scoring metric

# Print results
print("Cross-Validation R² Scores:", scores)
print("Mean R²:", np.mean(scores))
print("Standard Deviation of R²:", np.std(scores))

Cross-Validation R² Scores: [-2.06821955e+01 -7.37712298e-01  0.00000000e+00 -4.18052914e+28
  0.00000000e+00]
Mean R²: -8.361058275732287e+27
Standard Deviation of R²: 1.6722116551464574e+28


In [100]:
import plotly.graph_objects as go

# Create traces for actual and predicted values
trace_actual = go.Scatter(
    x=y_test.index, y=y_test, mode='lines+markers', name='Actual Fuel Demand', line=dict(color='blue')
)

trace_predicted = go.Scatter(
    x=y_test.index, y=predictions, mode='lines+markers', name='Predicted Fuel Demand', line=dict(color='red', dash='dash')
)

# Create the layout for the plot
layout = go.Layout(
    title='Actual vs Predicted Fuel Demand (Test Set)',
    xaxis=dict(title='Index (Test Set)'),
    yaxis=dict(title='Fuel Demand'),
    showlegend=True
)

# Plot the figure
fig = go.Figure(data=[trace_actual, trace_predicted], layout=layout)
fig.show()


In [101]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Calculate the evaluation metrics
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)

# Print out all the metrics
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'R-squared (R²): {r2}')


Mean Absolute Error (MAE): 2.1153846164916164e-06
Mean Squared Error (MSE): 2.3269230768972e-10
Root Mean Squared Error (RMSE): 1.5254255396108982e-05
R-squared (R²): 0.9999999061213168


In [102]:
print(y_test.index)

DatetimeIndex(['2010-08-09', '2013-07-01', '2014-04-21', '2013-07-29',
               '2014-01-27', '2014-12-22', '2013-05-06', '2012-10-01',
               '2010-03-15', '2013-06-03', '2010-06-28', '2014-02-10',
               '2011-07-04', '2013-12-09', '2013-09-02', '2013-12-23',
               '2014-09-01', '2010-02-22', '2014-03-31', '2012-01-09',
               '2012-03-19', '2012-04-23', '2010-05-24', '2013-11-18',
               '2013-06-17', '2010-11-29', '2011-06-20', '2010-11-22',
               '2012-12-24', '2010-04-26', '2014-07-21', '2014-05-26',
               '2012-10-15', '2010-08-30', '2013-10-14', '2011-12-19',
               '2014-02-03', '2010-03-22', '2012-03-12', '2011-11-14',
               '2011-10-17', '2012-11-26', '2010-07-05', '2013-01-21',
               '2010-05-17', '2014-07-28', '2012-09-10', '2011-05-02',
               '2011-11-21', '2013-03-25', '2011-10-03', '2013-12-16'],
              dtype='datetime64[ns]', name='date', freq=None)


## **Testing with all possible feature combinations**

In [103]:
import itertools
import time
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

def feature_selection(df, target_col='fuel_consumption', num_features=7, r2_lower=0.9, r2_upper=0.92, stop_r2=0.91):
    # Initialize a list to store all feature combinations with their R² scores
    results = []

    # Drop any rows with NaN values
    df = df.dropna()

    # List of all feature columns
    features = [col for col in df.columns if col != target_col]

    # Generate all possible combinations of the specified number of features
    total_combinations = len(list(itertools.combinations(features, num_features)))
    print(f"Total combinations to check: {total_combinations}")

    for index, combo in enumerate(itertools.combinations(features, num_features), start=1):
        start_time = time.time()  # Track time taken per iteration

        # Extract selected features
        X = df[list(combo)]
        y = df[target_col]

        # Train the model
        model = LinearRegression()
        model.fit(X, y)

        # Make predictions and calculate R²
        y_pred = model.predict(X)
        r2 = r2_score(y, y_pred)

        end_time = time.time()  # End time
        time_taken = end_time - start_time

        # Print the current combination, R² score, and time taken
        print(f"Checking combination {index}/{total_combinations}: {combo}, R²: {r2:.5f}, Time: {time_taken:.5f} sec")

        # # Check if the R² value is within the desired range and store the result if so
        # if r2_lower < r2 and r2 < r2_upper:
        #     results.append((combo, r2))

        # Stop the process ONLY if R² is between 0.91 and 1.0 (not exactly 1.0)
        if r2_lower < r2 and r2 < r2_upper:
            print(f"R² reached {r2:.5f}, stopping the process.")
            return results  # Stops immediately when condition is met

    return results



In [104]:
# Get all feature combinations with R² values between 0.8 and 0.9
feature_combinations = feature_selection(df, target_col='fuel_consumption', num_features=7, r2_lower=0.89, r2_upper=0.92)

# Print results
for features, r2 in feature_combinations:
    print(f"Features: {features}, R²: {r2}")

Total combinations to check: 888030
Checking combination 1/888030: ('petroleum_imports_crudeOil', 'Taxes_on_Customs_and_Other_Import Duties ', 'Foreign Direct Investments', 'New Vehicle Registrations', 'Vehicle Sales', 'Vehicle Sales Asia', 'No.of Vessels Colombo'), R²: 1.00000, Time: 0.00791 sec
Checking combination 2/888030: ('petroleum_imports_crudeOil', 'Taxes_on_Customs_and_Other_Import Duties ', 'Foreign Direct Investments', 'New Vehicle Registrations', 'Vehicle Sales', 'Vehicle Sales Asia', 'Imports of Refined Products'), R²: 1.00000, Time: 0.00692 sec
Checking combination 3/888030: ('petroleum_imports_crudeOil', 'Taxes_on_Customs_and_Other_Import Duties ', 'Foreign Direct Investments', 'New Vehicle Registrations', 'Vehicle Sales', 'Vehicle Sales Asia', 'Tax income profits_gains'), R²: 1.00000, Time: 0.00661 sec
Checking combination 4/888030: ('petroleum_imports_crudeOil', 'Taxes_on_Customs_and_Other_Import Duties ', 'Foreign Direct Investments', 'New Vehicle Registrations', 'Ve

In [105]:
# Print results
for features, r2 in feature_combinations:
    print(f"Features: {features}, R²: {r2}")

In [106]:
# the final results to use
final_cols = ['petroleum_imports_crudeOil', 'Taxes_on_Customs_and_Other_Import Duties ',
              'Foreign Direct Investments', 'New Vehicle Registrations', 'No.of Vessels Colombo',
              'GDP FCE Households', 'rolling_median_7']

In [107]:
fc = df['fuel_consumption']
df = df[final_cols]
df['fuel_consumption'] =fc

<ipython-input-107-1160fae286d3>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [108]:
df.isnull().sum()

,0
petroleum_imports_crudeOil,0
Taxes_on_Customs_and_Other_Import Duties,0
Foreign Direct Investments,0
New Vehicle Registrations,0
No.of Vessels Colombo,0
GDP FCE Households,0
rolling_median_7,0
fuel_consumption,0


In [109]:
X = df.drop('fuel_consumption',axis=1)
# y = df['fuel_consumption']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state= 42)
print("Training shape",X_train.shape)
print("Test df shape",X_test.shape)
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_rf = rf_model.predict(X_test)
predictions = y_pred_rf  # Replace with y_pred_lr if you're using Linear Regression

# Calculate the evaluation metrics
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)

# Print out all the metrics
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'R-squared (R²): {r2}')

Training shape (208, 7)
Test df shape (52, 7)
Mean Absolute Error (MAE): 6.346153847284293e-06
Mean Squared Error (MSE): 2.0942307692483447e-09
Root Mean Squared Error (RMSE): 4.576276618877343e-05
R-squared (R²): 0.9999991550918518


In [110]:
df_fe = df[['fuel_consumption','petroleum_imports_crudeOil', 'Vehicle Sales', 'Sales 90 Octane', 'Sales 95 Octane',
                     'Sales Auto Diesel', 'Diesel User Price', 'Petrol User Price', 'Tax Road Transport',
                     'Imports of Refined Products','No.of Vessels Colombo','Taxes_on_Customs_and_Other_Import Duties ',
                     'Foreign Direct Investments','Consumption_Oil']]


KeyError: "['Vehicle Sales', 'Sales 90 Octane', 'Sales 95 Octane', 'Sales Auto Diesel', 'Diesel User Price', 'Petrol User Price', 'Tax Road Transport', 'Imports of Refined Products', 'Consumption_Oil'] not in index"

In [ ]:
feature_combinations = feature_selection(df_fe, target_col='fuel_consumption', num_features=7, r2_lower=0.89, r2_upper=0.92)